<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td>
            <h2 style="color:#32CD32;">Intro</h2>
            <span style="color:#32CD32;">
                This is a cover letter builder that uses agentic AI to help you create cover letters. This is a notebook, where you can experiment with different promts, models, or design patterns. You can run each cell individualy, or use the Run All button at the top. 
            </span>
        </td>
    </tr>
</table>

In [1]:
from pypdf import PdfReader
from cover_letter_builder import CoverLetterBuilder

c:\Code\Work Projects\projects\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# This app uses the default values.
# You can just run it, but it won't be personalized to you.
CoverLetterBuilder()

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td>
            <span style="color:#32CD32;">
                You can already run the CoverLetterBuilder, but it only has the default values right now. You can replace the default values, or you can pass in parameters to replace them. Let's work on that.
            </span>
        </td>
    </tr>
</table>

In [2]:
# models used
creator_model = "gpt-4o"
evaluator_model = "o4-mini"

In [3]:
# Path to your summary and cover_letter
summary_path = "about/summary.txt"
cover_letter_path = "about/cover_letter_template.txt"

# path to your resume
resume_path = "about/resume.pdf"



In [4]:
# Insert your name here. 
name = "Charles McTurland"

In [5]:
 # evalion limit - you can limit it to avoid expences
eval_limit = 10

In [6]:
# if you want to include AI suggestions on how to further improve the cover letter, set this to true
include_feedback = True

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td>
            <span style="color:#32CD32;">
                You provided all of the personalized information to the cover_letter_builder, and now its ready to create a resume specificaly for you! Let's try it out. 
            </span>
        </td>
    </tr>
</table>

In [7]:
# This app uses all of the input that we just set up. 
CoverLetterBuilder(
    creator_model = creator_model, 
    evaluator_model = evaluator_model, 
    name = name, 
    eval_limit = eval_limit,
    summary_path = summary_path,
    cover_letter_path = cover_letter_path,
    resume_path = resume_path,
    include_feedback=include_feedback)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


error
Failed to scrape job posting
Failed evaluation - retrying
## Cover Letter:
Dear Hiring Manager,

I am writing to express my interest in the Computer Scientist – Mid to Senior Level position with Bow Wave LLC, as advertised for the Drug Enforcement Agency (DEA). With a solid background in building scalable applications and commitment to excellence in programming, I am excited about the opportunity to contribute to Bow Wave's impactful work in national security services.

At Embark, where I have been since 2015, I played a pivotal role in transforming a legacy application into a high-performing single-page app, significantly boosting the company's annual revenue. This experience has honed my skills in collaborating with cross-functional teams and leading complex projects, aligning well with the DEA role’s requirements for system development life cycles, leadership, and customer service focus. My proficiency with SQL databases and cloud infrastructure, supported by knowledge of MSFT

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td>
            <span style="color:#32CD32;">
                Hey, it works! But we can take it a bit further. You can further cusomize the app by changing the promts for the AI.  
            </span>
        </td>
    </tr>
</table>

In [ ]:
# Reads your summary and cover letter.
with open(summary_path, "r", encoding="utf-8") as f:
    summary = f.read()

with open(cover_letter_path, "r", encoding="utf-8") as f:
    cover_letter_template = f.read()
    
# Reads you resume. 
reader = PdfReader(resume_path)
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [7]:
# Initial system prompt: you can adjust it to your needs and preferences. There is always room for improvement!
system_prompt = f"""You are a proffesional cover letter writer, and your job is to write a cover letter for {name}, highlighting {name}'s skills, experience, and achievements. 
particularly questions related to {name}'s career, background, skills and experience. 
Your responsibility is to represent {name} in the letter as faithfully as possible. 
You are given a summary of {name}'s background and Resume which you can use in the cover letter. 
You are given an example of a cover letter from {name}. Try and use a similar language and style. Do NOT include the placeholder information in the cover letter. 
Be professional and engaging, uing the tone and style suitable for a cover letter.
Do not make up any information, and only use the information provided.
Don't be too verbose, and use a 3 paragraph format.
Respond with a cover letter and nothing else.
Do not include the address or contact information. 
You will be given a job description, and you will need to tailor the cover letter to the job description.
You will be evaluated, and if evalutor decides that your cover letter is not up to standart, you will be given your previus cover letters and feedback on them. 
You have to listen to the feedback, and improve your cover letter accordingly to the feedback.
\n\n## Summary:\n{summary}\n\n## Resume:\n{resume}\n\n ## Cover Letter Template:\n{cover_letter_template}\n\n
"""

In [8]:
# Evaluator prompt: you can adjust it to your needs and preferences. There is always room for improvement!
evaluator_prompt = f"""
You are a professional evaluator that decides whether a cover letter is acceptable. 
You are provided with {name}'s summary and resume, an example of a cover letter from {name}, the job description, and the cover letter. 
Your task is to evaluate the cover letter, and reply with whether it is acceptable and your feedback. 
You need to ensure if the cover letter is professional, engaging, and tailored to the job description. 
You need to ensure if the cover letter was likely made by AI, and if it was made by AI, deny it, and provide feedback. Do not allow AI generated cover letters.
You need to ensure that the cover letter has a strong and engaging opening paragraph. 
You need to ensure that the cover letter is concise and uses the standard 3 paragraph format.
Here's the information:
\n\n## Summary:\n{summary}\n\n## Resume:\n{resume}\n\n## Cover Letter Template:\n{cover_letter_template}\n\n
With this context, please evaluate the cover letter, replying with whether the cover letter is acceptable and your feedback.
"""

In [ ]:
# This app uses the prompts that we just set up. 
CoverLetterBuilder(
    creator_model = creator_model, 
    evaluator_model = evaluator_model, 
    eval_limit = eval_limit,
    include_feedback=include_feedback,
    system_prompt = system_prompt,
    evaluator_prompt = evaluator_prompt)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td>
            <h2 style="color:#32CD32;">Conclusion</h2>
            <span style="color:#32CD32;">
                And thats it! At least for now. The only thing left to do is to run application, and start generating cover letters using your new tool. You can customize it by insering as many or as view of the optional parameters. 
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td>
            <span style="color:#32CD32;">
                Here is an example of a cover letter written by the AI:
            </span>
        </td>
    </tr>
</table>

Dear Hiring Manager,

I am writing to express my enthusiasm for the Software Engineer position at T-Mobile, located in Bellevue, WA. The opportunity to contribute to T-Mobile's renowned Un-carrier initiative and to advance its cutting-edge 5G network aligns closely with my passion for innovative technology solutions and extensive experience in developing scalable web applications.

My background at Embark, where I successfully re-architected a legacy application into a high-performing single-page app with a direct impact on increasing revenue by $1.4M annually, has honed my skills in both back-end and front-end technologies. I possess extensive experience utilizing Java and C# for developing REST microservices, as well as jQuery, HTML, and CSS for creating intuitive user interfaces. My proficiency in SQL Server for data management, alongside my use of Swagger for API documentation, directly complements T-Mobile's technical requirements.

T-Mobile's dedication to technological progress and user experience speaks to my professional values. I am adept at utilizing tools such as Splunk for reporting and monitoring, as well as Jenkins and GitLab for seamless continuous integration and deployment. I am particularly excited about the opportunity to contribute within T-Mobile's Agile and collaborative environment, leveraging my expertise to drive innovation and connectivity.

Thank you for considering my application. I am eager to discuss how my skills and experience can contribute to T-Mobile's continued success and its mission to redefine the wireless industry. I look forward to the opportunity to connect.

Sincerely,
Charles McTurland